---
title: "ClassyDSPy: A Better DSPy API"
format: html
execute:
  echo: true
  eval: true
---


## Overview

This document explores an improved API design for DSPy that addresses key pain points:

1. **No DSL for signatures** - Use Pydantic models instead of InputField/OutputField
2. **No prompts in docstrings** - Separate prompt definition from code
3. **Standard Python patterns** - Familiar APIs that don't require learning custom syntax

## Core Design Philosophy

The API should be built around:
- **Input schema** (Pydantic model)
- **Output schema** (Pydantic model) 
- **Input-target examples** (type-checked to match schemas)
- **Prompt templates** (separate from code)

## Basic Example: Person Extraction

```{{python}}
from pydantic import BaseModel
import dspy

class Person(BaseModel):
    name: str
    age: int
    occupation: str

person = dspy.Predict(input_format=str,
                     output_format=Person,
                     instructions="Extract:",
                     lm=dspy.LM("openai/gpt-4o-mini"))

print(person("John is a 30-year-old software engineer"))
# Person(name='John', age=30, occupation='software engineer')
```

## Advanced API: Compilation and Optimization


In [ ]:
# Example of how .compile and .save should work
# with input/output schemas and training examples

# Define schemas
class PersonInput(BaseModel):
    text: str
    
class PersonOutput(BaseModel):
    name: str
    age: int
    occupation: str
    confidence: float

# Training examples (type-checked)
examples = [
    (PersonInput(text="Alice works as a doctor and is 35"), 
     PersonOutput(name="Alice", age=35, occupation="doctor", confidence=0.95)),
    # ... more examples
]

# Compile with optimization
predictor = dspy.Predict(
    input_schema=PersonInput,
    output_schema=PersonOutput,
    prompt_template="Extract person information from: {text}"
)

# Optimize with examples
optimized = predictor.compile(
    examples=examples,
    optimizer=dspy.BootstrapFewShot(),
    evaluation_metric="accuracy"
)

# Save for reuse
optimized.save("person_extractor.json")

## Evaluation Dimensions

For complex tasks, evaluation may need multiple dimensions:


In [ ]:
from typing import List

class EvaluationDimension(BaseModel):
    name: str
    description: str
    weight: float = 1.0

class TaskOutput(BaseModel):
    result: str
    accuracy_score: float
    creativity_score: float
    coherence_score: float

# Multi-dimensional evaluation
evaluations = [
    EvaluationDimension(
        name="accuracy", 
        description="Factual correctness of extracted information",
        weight=0.5
    ),
    EvaluationDimension(
        name="creativity", 
        description="Novel insights and connections made",
        weight=0.3
    ),
    EvaluationDimension(
        name="coherence", 
        description="Logical flow and consistency",
        weight=0.2
    )
]

predictor = dspy.Predict(
    input_schema=PersonInput,
    output_schema=TaskOutput,
    evaluation_dimensions=evaluations
)

## Fine-tuning and Reward Functions

Beyond prompt optimization, the API should support:


In [ ]:
# Fine-tuning support
fine_tuned = predictor.fine_tune(
    examples=training_data,
    base_model="openai/gpt-4o-mini",
    epochs=3,
    learning_rate=1e-5
)

# Reward function optimization
reward_optimized = predictor.optimize_with_reward(
    reward_function=lambda input, output: calculate_reward(input, output),
    rl_algorithm="ppo",
    iterations=1000
)

## Benefits of This Approach

1. **Type Safety**: Pydantic provides runtime type checking and IDE support
2. **Familiar Patterns**: Standard Python class definitions instead of DSL
3. **Separation of Concerns**: Prompts, schemas, and logic are clearly separated
4. **Composability**: Easy to combine and extend components
5. **Tooling Support**: Works with existing Python tooling (linters, formatters, etc.)